In [29]:
## ----------------------------------------------------------------------------
## Course: ECON 601
## Instructor: Michael Magill
## Problem: 1
## Author: Noah Miller
## Date: 09/01/2017
## Desciption: Solving 3 agent 1 good Arrow-Debreu Equilibrium
## ----------------------------------------------------------------------------



## Setup ------------------------------------------------------------------   
    #Pkg.add("JuMP")
    #Pkg.add("Ipopt")
    #Pkg.update()

    # import necessary libraries
using JuMP, Ipopt



## create ADEQ function for solving
function ADEQ(α, a1, a2, a3, δ1, δ2, δ3, ρ1, ρ2, ρ3, β1, β2, β3, ϵ1, ϵ2, ϵ3)
    
        ## set model and solve method -------------------------------------------- (1)
        ADEQ = Model(solver=IpoptSolver(print_level=0))


        
        ## create endogenous variables ------------------------------------------- (2)

            # endowment variables (2.1)
        ω1 = [0.0, β1 + ϵ1, β1 + ϵ1, β1 - ϵ1, β1 - ϵ1]
        ω2 = [0.0, β2 + ϵ2, β2 - ϵ2, β2 + ϵ2, β2 - ϵ2]
        ω3 = [β3, 0.0, 0.0, 0.0, 0.0]

            # price variables (2.2)
        boundary = 1e-9
        @variable(ADEQ, π[1:5] >= boundary)
        JuMP.fix(π[1], 1.0)

            # consumption variables (2.3)
        @variables ADEQ begin    
            x1[1:5] >= 0.0
            x2[1:5] >= 0.0
            x3[1:5] >= 0.0   
        end


        
        ## set individual utility functions -------------------------------------- (3)
        @NLexpressions ADEQ begin     
            # agent 1 utility (3.1.1)
            u1, (1/(1-α))*(a1+x1[1])^(1-α)+δ1*((ρ1[2]*((1/(1-α))*(a1+x1[2])^(1-α)))+(ρ1[3]*((1/(1-α))*(a1+x1[3])^(1-α)))+(ρ1[4]*((1/(1-α))*(a1+x1[4])^(1-α)))+(ρ1[5]*((1/(1-α))*(a1+x1[5])^(1-α))))

            # agent 2 utility (3.1.2)
            u2, (1/(1-α))*(a2+x2[1])^(1-α)+δ2*((ρ2[2]*((1/(1-α))*(a2+x2[2])^(1-α)))+(ρ2[3]*((1/(1-α))*(a2+x2[3])^(1-α)))+(ρ2[4]*((1/(1-α))*(a2+x2[4])^(1-α)))+(ρ2[5]*((1/(1-α))*(a2+x2[5])^(1-α))))

            # agent 3 utility (3.1.3)
            u3, (1/(1-α))*(a3+x3[1])^(1-α)+δ3*((ρ3[2]*((1/(1-α))*(a3+x3[2])^(1-α)))+(ρ3[3]*((1/(1-α))*(a3+x3[3])^(1-α)))+(ρ3[4]*((1/(1-α))*(a3+x3[4])^(1-α)))+(ρ3[5]*((1/(1-α))*(a3+x3[5])^(1-α))))
        end

            # (3.1x)
            ## Although cleaner than above, solver currently doesn't contain support for nested ∑ function within constraints
            #@NLexpressions ADEQ begin 
            #    u1, (1/(1-α))*(a1+x1[1])^(1-α)+δ1*∑(ρ1[i]*((1/(1-α))*(a1+x1[i])^(1-α)) for i in 2:5)
            #    u2, (1/(1-α))*(a2+x2[1])^(1-α)+δ2*∑(ρ2[i]*((1/(1-α))*(a2+x2[i])^(1-α)) for i in 2:5)
            #    u3, (1/(1-α))*(a3+x3[1])^(1-α)+δ3*∑(ρ3[i]*((1/(1-α))*(a3+x3[i])^(1-α)) for i in 2:5)
            #end


        
        ## set objective function ------------------------------------------------ (4)
        @NLobjective(ADEQ, Max, (u1+u2+u3))



        ## set constraints ------------------------------------------------------- (5)

            # endowment/consumption (5.1)
        @expressions ADEQ begin  
            # endowment (5.1.1)
            ∑ω1, ∑(π[i]*ω1[i] for i in 1:5)    # agent 1 endowment
            ∑ω2, ∑(π[i]*ω2[i] for i in 1:5)    # agent 2 endowment
            ∑ω3, ∑(π[i]*ω3[i] for i in 1:5)    # agent 3 endowment

            # consumption (5.1.2)
            ∑x1, ∑(π[i]*x1[i] for i in 1:5)    # agent 1 consumption
            ∑x2, ∑(π[i]*x2[i] for i in 1:5)    # agent 2 consumption
            ∑x3, ∑(π[i]*x3[i] for i in 1:5)    # agent 3 consumption    
        end

            # market clearing expressions (5.2)
        @expressions ADEQ begin
            mkt1, (x1[2]+x2[2]+x3[2])-(ω1[2]+ω2[2]+ω3[2])    # state 1
            mkt2, (x1[3]+x2[3]+x3[3])-(ω1[3]+ω2[3]+ω3[3])    # state 2
            mkt3, (x1[4]+x2[4]+x3[4])-(ω1[4]+ω2[4]+ω3[4])    # state 3
            mkt4, (x1[5]+x2[5]+x3[5])-(ω1[5]+ω2[5]+ω3[5])    # state 4
                                                             # state 0 not needed by Walras Thrm
        end

            # (5.2x)
            ## Although cleaner, this loop doesn't seem to work
            #mkts = [mkt1, mkt2, mkt3, mkt4]
            #for (i, mkt) in enumerate(mkts)
            #    @expression(ADEQ, mkt, (x1[i]+x2[i]+x3[i])-(ω1[i]+ω2[i]+ω3[i]))
            #end

            # set constraints (5.3)
        @constraints ADEQ begin
            # consumption constraints (5.3.1)
            ∑x1 <= ∑ω1      #agent 1
            ∑x2 <= ∑ω2      #agent 2
            ∑x3 <= ∑ω3      #agent 3

            # market clearing constraints (5.3.2)
            mkt1 == 0.0     #state 1
            mkt2 == 0.0     #state 2  
            mkt3 == 0.0     #state 3
            mkt4 == 0.0     #state 4
                            #state 0 not included by Walras Thrm
        end


        
        ## solve model and print results ----------------------------------------- (6)

            #print(ADEQ) [uncomment if want full printout of model setup] (6.1)

            # solve model (6.2)
        result = solve(ADEQ)
        
            # store results (6.3)
        #X = [[getvalue(x1[1]) getvalue(x1[2]) getvalue(x1[3]) getvalue(x1[4]) getvalue(x1[5]) getvalue(∑x1)], 
        #     [getvalue(x2[1]) getvalue(x2[2]) getvalue(x2[3]) getvalue(x2[4]) getvalue(x2[5]) getvalue(∑x1)], 
        #     [getvalue(x3[1]) getvalue(x3[2]) getvalue(x3[3]) getvalue(x3[4]) getvalue(x3[5]) getvalue(∑x1)]]
        #Π = [getvalue(π[1]), getvalue(π[2]), getvalue(π[3]), getvalue(π[4]), getvalue(π[5])]
    #return ADEQ, X, Π
    
        ## display state-based and total consumption for each agent (7)
        println("")
        println("Agent 1's consumption in s0 is: ", X[1][1])
        println("                      in s1 is: ", X[1][2]) 
        println("                      in s2 is: ", X[1][3]) 
        println("                      in s3 is: ", X[1][4])
        println("                      in s4 is: ", X[1][5])
        println("                      in ∑ is:  ", X[1][6])

        println("")
        println("Agent 2's consumption in s0 is: ", X[2][1])
        println("                      in s1 is: ", X[2][2]) 
        println("                      in s2 is: ", X[2][3]) 
        println("                      in s3 is: ", X[2][4])
        println("                      in s4 is: ", X[2][5])
        println("                      in ∑ is:  ", X[2][6])

        println("")
        println("Agent 3's consumption in s0 is: ", X[3][1])
        println("                      in s1 is: ", X[3][2]) 
        println("                      in s2 is: ", X[3][3]) 
        println("                      in s3 is: ", X[3][4])
        println("                      in s4 is: ", X[3][5])
        println("                      in ∑ is:  ", X[3][6])

        ## display price values (2)
        println("")
        println("π0 = ", Π[1])
        println("π1 = ", Π[2])
        println("π2 = ", Π[3])
        println("π3 = ", Π[4])
        println("π4 = ", Π[5])
end

ADEQ (generic function with 1 method)

In [30]:
## assign exogenous parameter variables ----------------------------------- 
#  β1 = 500.0
#  β2 = 500.0
#  β3 = 1000.0
#  ϵ1 = 0.0
#  ϵ2 = 0.0
#  ϵ3 = 0.0
#  α = 2.0
#  a1 = 0.0
#  a2 = 0.0
#  a3 = 0.0
#  δ1 = 0.95
#  δ2 = 0.95
#  δ3 = 0.95
#  ρ1 = [1.0, 0.25, 0.25, 0.25, 0.25]
#  ρ2 = [1.0, 0.25, 0.25, 0.25, 0.25]
#  ρ3 = [1.0, 0.25, 0.25, 0.25, 0.25]
# function ADEQ(α, a1, a2, a3, δ1, δ2, δ3, ρ1, ρ2, ρ3, β1, β2, β3, ϵ1, ϵ2, ϵ3)
ADEQ(2.0, 0.0, 0.0, 0.0, 0.95, 0.95, 0.95, [1.0, 0.25, 0.25, 0.25, 0.25], [1.0, 0.25, 0.25, 0.25, 0.25], [1.0, 0.25, 0.25, 0.25, 0.25], 500.0, 500.0, 1000.0, 0.0, 0.0, 0.0)


Agent 1's consumption in s0 is: 333.3329531187592
                      in s1 is: 333.3336832387324
                      in s2 is: 333.3336832387324
                      in s3 is: 333.3336832387324
                      in s4 is: 333.3336832387325
                      in ∑ is:  1000.0014883711511

Agent 2's consumption in s0 is: 333.3329531187592
                      in s1 is: 333.3336832387324
                      in s2 is: 333.3336832387324
                      in s3 is: 333.3336832387324
                      in s4 is: 333.3336832387324
                      in ∑ is:  1000.0014883711511

Agent 3's consumption in s0 is: 333.3321328742983
                      in s1 is: 333.33263352253516
                      in s2 is: 333.33263352253516
                      in s3 is: 333.3326335225352
                      in s4 is: 333.33263352253516
                      in ∑ is:  1000.0014883711511

π0 = 1.0
π1 = 0.5000008765802981
π2 = 0.500000876580356
π3 = 0.5000008765802989
π4 = 0.500

ADEQresults (generic function with 1 method)